# HW1 - Intro to the Map Reduce Paradigm.  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarassingly Parallel" task.
* ... __estimate__ the runtime of embarassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. __Please refer to the `README` for detailed submission instructions__.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 2.7
!python --version

Python 3.5.2 :: Anaconda custom (64-bit)


In [2]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [3]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

> My name is Stanimir Vichev. I currently live and work in London, although I am originally from Bulgaria. This is my fourth term in MIDS, and I am hoping to finish MIDS by September 2018. This module is one of the main reasons I am doing MIDS, since it bridges the gap between machine learning and big data, which is something I am very interested in and might need in the future in my current job. 

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

> A simpler model that tries to fit data has high bias, since it has low accuracy when fitting the training data. It also has low variance because if we train it multiple times on different samples from the dataset, the resulting functions wouldn't vary much in their parameters, meaning that they would fit a random sample of the data better. As a model becomes more complicated, bias decreases, since it is better able to fit the training data. However, its complexity means that it is worse at generalizing to new data, hence its variance across datasets increases. This leads to the bias-variance trade-off, since aiming to decrease one necessarily increases the other. As such, a model's error can be decomposed into a bias element and a variance element:

> Bias: $E[h(x^{*})]-f(x^{*})$ 

> Variance: $E[(h(x^{*})-E[h(x^{*})])^2]$

> When training a model in machine learning, we can look at the bias (how well model performs on a known dataset) to determine whether or not our model is too simple for the data provided. 

# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example you might choose to remove punctuation, or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week) we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the total number of words. In the space provided below give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation, make everything lowercase, and discard numerical digits. Suppose __`tokenize(x)`__ is a Python function that peforms the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]


* __d) unittest:__ Run the provided unittest to demonstrate that your regular expression works.

### Q3 Student Answers:
> __a)__ If we tokenize the sentence "Jack had to buy eggs, bacon, and bread, remembering that the eggs should be free-range." using just spaces, we would count "eggs," and "eggs" as two separate words, appearing once in each sentence. If instead we tokenize that same sentence using spaces and commas, we would count "eggs" twice correctly, so it would have a higher likelihood.

> __b)__ ["by","the","bye","what", "became", "of", "alice", "s", "hats"]

In [10]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("[a-z]+")          

In [11]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


In [ ]:
# Part D - unittest - SKIP THIS CELL
# NOTE: this term unittests are still being developed, we appreciate your patience.
# Please visually confirm that your code works and then move on to the next section.

# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [13]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl 'http://www.gutenberg.org/files/11/11-0.txt' -o data/alice.txt

curl: (1) Protocol "'https" not supported or disabled in libcurl


In [14]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

'head' is not recognized as an internal or external command,
operable program or batch file.


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [15]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Writing data/alice_test.txt


In [16]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

'ls' is not recognized as an internal or external command,
operable program or batch file.


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) short response:__ How is a `defaultdict` different than a regular Python dictionary? Why might a `defaultdict` be particularly convenient in the context of counting word occurrences as you stream over a text file? (`HINT:` _if this Python type is new to you, check out the [documentation](https://docs.python.org/3.3/library/collections.html#defaultdict-objects) and [this brief tutorial](https://www.accelebrate.com/blog/using-defaultdict-python/)_)


* __b) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __c) testing:__ Run the cell marked `part c` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __d) results:__ When you are confident in your implementation, run the cell marked `part d` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __e) short reponse:__ Suppose, after running your analysis, you decide that you'd really like  a word and its plural (eg. 'hatter' and 'hatters') to be counted as the same word. Would it be more efficient to post-process your output file or to change your tokenizing and re-run the original word count 'analysis'? Explain your reasoning briefly.

### Q4 Student Answers:
> __a)__ A defaultdict is initialized in a way that means new keys automatically have a default value. If we want to count words, and we have created myDict = defaultdict(lambda: 0), this means that we can call myDict[newWord] += 1 without having to do myDict[newWord] = 0 first.

> __b-c)__ _Complete the coding portions of this question before answering 'e'._

> __e)__ Type your answer here!

In [ ]:
# part b - DO YOUR WORK IN wordCount.py

In [24]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

has 	 1
lines 	 1
file 	 3
small 	 3
a 	 2
for 	 1
two 	 1
test 	 3
is 	 2
this 	 3


In [25]:
# part d - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [26]:
!head data/alice_counts.txt

'head' is not recognized as an internal or external command,
operable program or batch file.


__Check your results:__ How many times does the word "alice" appear in the book? 

In [ ]:
# EXPECTED OUTPUT: 403
!grep alice data/alice_counts.txt

__Check your results:__ How many times does the word "hatter" appear in the book? 

In [ ]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

__Check your results:__ How many times does the word "queen" appear in the book? 

In [ ]:
# EXPECTED OUTPUT: 75
!grep queen data/alice_counts.txt

# Question 5: Unix Sorting
Another common task in this course's assignments will be to make strategic use of sorting. To illustrate what we mean by 'strategic,' let's return to the (admitedly artificial) scenario posed in Question 4e): _we have a file of word counts but we want to find all pairs of words and plurals (eg. 'queen' and 'queens') to join their counts into a single record_. Suppose we were going to do this by hand. A logical approach would be to start with the first word and read through each subsequent word to check if they are a singular-plural pair, then repeat this process for each word in the file.

### Q5 Tasks:
* __a) short response + code:__ In the worst case scenario, how many comparisons will we have to make to perform the process described above? Use $\LaTeX$ formatting to show the calculations that led to your answer. What is the Big O complexity of this "algorithm"? [`HINTs:` _To answer the first part of this question you'll want to know how many words are in your_ `alice_counts.txt` _file -- in the space provided below, write a unix command to check. If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://cooervo.github.io/Algorithms-DataStructures-BigONotation/), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).]


* __b) short response:__ If the word count file were sorted alphabetically, how would your approach to this task change? How many comparisons would you have to make? What would the Big O complexity of this new strategy be?


* __c) short response:__ What is the Big O complexity of the fastest sorting algorithms? When we take the time to sort in to account, does it make sense to alphabetize before combining singular-plural pairs? [`HINT:` _Re: sorting algorithms -- Wikipedia is your friend, as is the cheatsheet from part a._]


* __d) very short response:__ What do we mean by 'strategic' when we say 'strategic sorting'?


* __e) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. pipe) the results to `data/alice_counts_A-Z.txt`. [`HINT:` if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)]


* __f) code:__ A (slightly) more likely scenario is that we'd want to look at the top 10 most frequent words that appear in the book. Write a unix command to sort your word count file from highest to lowest count. Save (i.e. pipe) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!

> __d)__ Type your answer here!

In [ ]:
# part a - write a unix command to check how many records are in your word count file


In [ ]:
# part e - unix command to sort your word counts alphabetically


In [ ]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

In [ ]:
# part f - unix command to sort your word counts from highest to lowest count


In [ ]:
# part f - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

<table>
<th>expected output for (e):</th>
<th>expected output for (f):</th>
<tr><td><pre>
a          690
abide      2
able       1
about      102
above      3
absence    1
absurd     2
accept     1
acceptance 1
accepted   2
</pre></td>
<td><pre>
the	 1818
and	 940
to	 809
a	 690
of	 631
it	 610
she	 553
i	 545
you	 481
said	 462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount.sh` takes an input file, splits it into a specified number of 'chunks' and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll pipe the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we'd have to add to `pWordCount.sh` to fix the problem.


* __c) code:__ We've provided a python script, `collateCounts.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount.sh` to make a one line modification so that it accepts `collateCount.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ Type your answer here!

In [ ]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount.sh
!chmod a+x wordCount.py

In [ ]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [ ]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

In [ ]:
# part c - make the collate script is executable (RUN THIS CELL AS IS)
!chmod a+x collateCounts.py

In [ ]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'collateCounts.py' \
                   > 'data/alice_pCounts.txt'

In [ ]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

### Q7 Tasks:

* __ a) short response:__ What is the potential scalability problem with the provided implementation of `collateCounts.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __ b) code:__ Fortunately a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount.sh` carefully then comment out your solution to `Q6.c` in `pWordCount.sh` and replace it with a line that alphabetically sorts the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Rewrite the main part of `collateCounts.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __ d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __ e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing. This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ Type your answer here!

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ Type your answer here!

> __e)__ Type your answer here!

In [ ]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'collateCounts.py'

In [ ]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'collateCounts.py' \
                   > 'data/alice_pCounts.txt'

In [ ]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

# Question 8: Scalability Considerations
In your reading for Week 1's live session, [Chapter1, section 2](../../Readings) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Lets try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4, 8, 16 and 32 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) BONUS:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spend on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!

> __d)__ Type your (OPTIONAL) answer here!

__Run the following cells to create the mapper referenced in `part b`__

In [ ]:
!mkdir demo

In [ ]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

In [ ]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [ ]:
%%timeit
!./pWordCount.sh 1 'data/alice.txt' 'demo/mapper.py'

In [ ]:
%%timeit
!./pWordCount.sh 2 'data/alice.txt' 'demo/mapper.py'

In [ ]:
%%timeit
!./pWordCount.sh 4 'data/alice.txt' 'demo/mapper.py'

__Run the following cells to repeat this process with your word count algorithm.__

In [ ]:
%%timeit
!./pWordCount.sh 1 'data/alice.txt' 'wordCount.py' 'collateCounts.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount.sh 2 'data/alice.txt' 'wordCount.py' 'collateCounts.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount.sh 4 'data/alice.txt' 'wordCount.py' 'collateCounts.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount.sh 8 'data/alice.txt' 'wordCount.py' 'collateCounts.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount.sh 16 'data/alice.txt' 'wordCount.py' 'collateCounts.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount.sh 32 'data/alice.txt' 'wordCount.py' 'collateCounts.py' > 'data/tmp'

# Question 9: Embarassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

__What do we mean when we call something 'Embarassingly Parallel'? Does this term describe a 'task'? an 'implementation of a task'? What makes word counting 'embarassingly parallel'?__ BONUS: define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Week 2 Async, particularly section 2.4 and the assigned_ [reading from EECS](https://patterns.eecs.berkeley.edu/?page_id=37) _about Map-Reduce Patterns_ ]

> Type your answer here!

# Question 10: Getting Set-up with Docker

Starting with next week's assignment we will be using a Docker container to ensure a consistent environment for the class. While it is not _required_ that you complete the assignments using this environment we cannot provide support for individual setups. We have done our best to provide a consistent experience. However there will inevitably be new issues that arise since the Machine Learning content in this course is closely intertwined with rapidly evolving infrastructure and since students come froma a range of backgrounds. We encourage you and your peers to help each other troubleshoot infrastructure challenges using the course slack channel and by submitting formal issues for our attention in the environment repo.

### Q10 Tasks:
* __a) set up Docker:__ If you haven't already follow the instructions in the the course infrastructure repository: https://github.com/UCB-w261/w261-environment to get set up. Once you're set up reopen this notebook on the container and run the provided cell to show that you have successfully set it up.


* __b) short response:__ At minimum, how much RAM must you allocate to your Docker container to ensure that Hadoop runs properly? [`HINT:` _the answer is in the environment repo's README.md_]


* __c) short response:__ What parameter must we add to our Hadoop streaming jobs if we want to use additional python packages inside our MapReduce or MRJob scripts? [`HINT:` _the answer is in the environment repo's README.md... no worries if you don't yet know what a Hadoop MapReduce job is yet._]


* __d) short response:__ What variable allows us to access the Spark Context from within a Jupyter Notebook running on the Docker container? [`HINT:` _the answer is in the environment repo's README.md... no worries if you don't yet know what a Spark Context is._]

### Q10 Student Answers:
> __b)__ Type your answer here!

> __c)__ Type your answer here!

> __d)__ Type your answer here!

In [ ]:
# part a - show us that you are on Docker! (RUN THIS CELL AS IS)
!ls -la / | grep docker

# Congratulations, you've completed HW1. 
# Please refer to the README for submission instructions.